# 6장 19대 대선 자료 분석하기
- 이 책을 집필완료한 후 선거관리 위원회 홈페이지가 재보궐선거로 구성화면이 변경되었습니다.
- 그러므로 일일이 진입하던 코드를 생략하고, 바로 19대 대통령 선거 결과를 확인할 수 있는 페이지 바로가기로 진입합니다.
- 그리고 6-1,6-2절의 코드는 바뀝니다. 그러나 6-3절부터 학습하시는 경우라면 Github에서 배포하는 데이터를 사용하시면 됩니다.
- 아래 6-1. 6-2절의 코드가 변경되는 것을 양해해 주시기 바랍니다.

## 6-1. Selenium과 Beatiful Soup을 이용한 데이터 획득 준비 작업

In [1]:
# 한글 지원 설정

import pandas as pd
import numpy as np

import platform
import matplotlib.pyplot as plt

%matplotlib inline

path = 'c:/Windows/Fonts/malgun.ttf'
from matplotlib import font_manager, rc
if platform.system() == 'Darwin' :
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows' :
    font_name = font_manager.FontProperties(fname = path).get_name()
    rc('font', family=font_name)
else :
    print('Unknown system... sorry~~~~')
    
plt.rcParams['axes.unicode_minus'] = False

In [2]:
from selenium import webdriver
import time

In [3]:
# 크롬 드라이버를 사용하여 크롬 창을 연다.
driver = webdriver.Chrome('driver/chromedriver.exe')

# nec.go.kr -> 분야별정보 -> 역대선거통계 -> 투계표 -> 개표현황(읍면동별)
driver.get('http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP04')

In [4]:
# '대통령선거'항목 클릭
driver.find_element_by_id('electionType1').click()

In [5]:
driver.find_element_by_id('electionName').send_keys('제19대')

In [6]:
driver.find_element_by_id('electionCode').send_keys('대통령선거')

In [7]:
# 'option'에 있는 시도 항목들을 전부 가지고 온다.

sido_list_raw = driver.find_element_by_xpath('//*[@id="cityCode"]') # 항목을 가져와야하기 때문에 xPath이용
sido_list = sido_list_raw.find_elements_by_tag_name('option')
sido_names_values = [option.text for option in sido_list]
sido_names_values = sido_names_values[1:]
sido_names_values

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [9]:
# 'option'에 있는 시도 항목들을 전부 가지고 온다.

sigun_list_raw = driver.find_element_by_xpath('//*[@id="townCode"]') # 항목을 가져와야하기 때문에 xPath이용
sigun_list = sigun_list_raw.find_elements_by_tag_name('option')
sigun_names_values = [option.text for option in sigun_list]
sigun_names_values = sigun_names_values[1:]
sigun_names_values

['종로구',
 '중구',
 '용산구',
 '성동구',
 '광진구',
 '동대문구',
 '중랑구',
 '성북구',
 '강북구',
 '도봉구',
 '노원구',
 '은평구',
 '서대문구',
 '마포구',
 '양천구',
 '강서구',
 '구로구',
 '금천구',
 '영등포구',
 '동작구',
 '관악구',
 '서초구',
 '강남구',
 '송파구',
 '강동구']

In [10]:
driver.find_element_by_xpath('//*[@id="searchBtn"]').click()

In [11]:
# beautifulsoup을 사용해서 현재 페이지 파싱
from bs4 import BeautifulSoup

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
#soup

In [12]:
# alignR을 모두 찾아서 1~4번째까지 출력
tmp = soup.find_all('td','alignR')
tmp[1:5]

# 투표수
# 문재인
# 홍준표
# 안철수

[<td class="alignR">102,566</td>,
 <td class="alignR">42,512</td>,
 <td class="alignR">22,325</td>,
 <td class="alignR">22,313</td>]

## 6-2. 19대 대선 개표 결과 데이터 획득하기

In [14]:
from tqdm import tqdm_notebook
# 시도/시군 목록을 넣기 위한 리스트를 생성해준다.
sido_name = []
sigun_name = []

# 시도 목록을 모두 입력칸에 반복해서 넣어준다.
for sido_value in tqdm_notebook(sido_names_values) :
    element = driver.find_element_by_id('cityCode')
    element.send_keys(sido_value)
    
    time.sleep(1)
    
    # 시군 목록을 모두 입력칸에 반복해서 넣어준다.
    sigun_list_raw = driver.find_element_by_xpath('//*[@id="townCode"]')
    sigun_list = sigun_list_raw.find_elements_by_tag_name('option')
    
    # 시군 목록에서 나온 값들을 출력
    sigun_names_values = [option.text for option in sigun_list]
    sigun_names_values = sigun_names_values[1:]
    
    # 시군 목록에서 나온 값들을 리스트에 추가해준다.
    for sigun_value in sigun_names_values :
        sido_name.append(sido_value)
        sigun_name.append(sigun_value)

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [15]:
# 각 값들을 데이터프레임으로 만들어 준다.
election_result = pd.DataFrame({'광역시도':sido_name, '시군':sigun_name})

election_result.head()

,광역시도,시군
0,서울특별시,종로구
1,서울특별시,중구
2,서울특별시,용산구
3,서울특별시,성동구
4,서울특별시,광진구


In [16]:
# 투표율을 얻기 위한 함수 생성
def get_vote_info(n) :
    # 페이지 html 소스를 읽어온다.
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 'td'태그에 있는 값들을 불러오고 ','를 삭제한다.
    tmp = soup.find_all('td','alignR')
    tmp_values = [float(tmp_val.get_text().replace(',','')) for tmp_val in tmp[1:5]]
    
    # 각 후보들의 득표 값들을 알맞게 넣어준다.
    pop[n]  = tmp_values[0] # 투표수
    moon[n] = tmp_values[1] # 문재인
    hong[n] = tmp_values[2] # 홍준표
    ahn[n]  = tmp_values[3] # 안철수

In [17]:
import numpy as np

# 값을 불러올 때, 없으면 nan처리한다.
def fail_procedure(n) :
    pop[n]  = np.nan
    moon[n] = np.nan
    hong[n] = np.nan
    ahn[n]  = np.nan

In [18]:
# nan처리 된 값들을 계산한다.
pop  = [np.nan] * len(election_result)
moon = [np.nan] * len(election_result)
hong = [np.nan] * len(election_result)
ahn  = [np.nan] * len(election_result)

len(pop), len(moon), len(hong), len(ahn)

(250, 250, 250, 250)

In [19]:
for n in tqdm_notebook(election_result.index) :
    # 에러가 나면 except 구분을 실행
    try :
        # 각 항목들에 값을 넣어주는 구문
        element = driver.find_element_by_id('cityCode')
        element.send_keys(election_result['광역시도'][n])
        
        time.sleep(0.5)
        
        element = driver.find_element_by_id('townCode')
        element.send_keys(election_result['시군'][n])
        
        driver.find_element_by_xpath('//*[@id="searchBtn"]').click()
        
        time.sleep(0.5)
        
        get_vote_info(n)
        
    except :
        print('---- Error ----')
        fail_procedure(n)

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----



In [20]:
election_result['pop'] = pop
election_result['moon'] = moon
election_result['ahn'] = ahn
election_result['hong'] = hong

election_result.head()

,광역시도,시군,pop,moon,ahn,hong
0,서울특별시,종로구,102566.0,42512.0,22313.0,22325.0
1,서울특별시,중구,82852.0,34062.0,19372.0,17901.0
2,서울특별시,용산구,148157.0,58081.0,32109.0,35230.0
3,서울특별시,성동구,203175.0,86686.0,45674.0,40566.0
4,서울특별시,광진구,240030.0,105512.0,52824.0,46368.0


In [22]:
# 에러가 난 부분들을 모아준다.
re_try_index = election_result[election_result['pop'].isnull()].index

for n in tqdm_notebook(re_try_index) :
    try :
        element = driver.find_element_by_id('cityCode')
        element.send_keys(election_result['광역시도'][n])
        
        time.sleep(0.5)
        
        element = driver.find_element_by_id('townCode')
        element.send_keys(election_result['시군'][n])
        
        driver.find_element_by_xpath('//*[@id="searchBtn"]').click()
        
        time.sleep(0.5)
        
        get_vote_info(n)
        
    except :
        print('---- Error ----')
        fail_procedure(n)

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----
---- Error ----



In [23]:
election_result['pop'] = pop
election_result['moon'] = moon
election_result['ahn'] = ahn
election_result['hong'] = hong

election_result.head()

,광역시도,시군,pop,moon,ahn,hong
0,서울특별시,종로구,102566.0,42512.0,22313.0,22325.0
1,서울특별시,중구,82852.0,34062.0,19372.0,17901.0
2,서울특별시,용산구,148157.0,58081.0,32109.0,35230.0
3,서울특별시,성동구,203175.0,86686.0,45674.0,40566.0
4,서울특별시,광진구,240030.0,105512.0,52824.0,46368.0


In [24]:
election_result.to_csv('data/06. election_result.csv', encoding = 'utf-8', sep=',')

## 6-3. 각 후보의 득표율과 지역 ID 정리하기

In [25]:
# csv 파일 읽어오기
election_result = pd.read_csv('data/06. election_result.csv', encoding='utf-8',index_col=0)
election_result.head()

,광역시도,시군,pop,moon,ahn,hong
0,서울특별시,종로구,102566.0,42512.0,22313.0,22325.0
1,서울특별시,중구,82852.0,34062.0,19372.0,17901.0
2,서울특별시,용산구,148157.0,58081.0,32109.0,35230.0
3,서울특별시,성동구,203175.0,86686.0,45674.0,40566.0
4,서울특별시,광진구,240030.0,105512.0,52824.0,46368.0


In [26]:
# '광역시도' 이름을 정리한다.
sido_candi = election_result['광역시도']
sido_candi = [name[:2] if name[:2] in ['서울','부산','대구','광주','인천','대전','울산']
                         else '' for name in sido_candi]

In [27]:
# 시구 이름을 정리하는 함수를 지정해준다.
def cut_char_sigu(name) :
    return name if len(name) == 2 else name[:-1]

In [29]:
import re

sigun_candi = ['']*len(election_result)

for n in election_result.index :
    each = election_result['시군'][n]
    if each[:2] in ['수원','성남','안양','안산','고양','용인','청주','천안','전주','포항','창원'] :
        # '시'글자를 기준으로 나눈다.
        sigun_candi[n] = re.split('시',each)[0] + ' ' + cut_char_sigu(re.split('시',each)[1])
    else :
        sigun_candi[n] = cut_char_sigu(each)
        

sigun_candi

['종로',
 '중구',
 '용산',
 '성동',
 '광진',
 '동대문',
 '중랑',
 '성북',
 '강북',
 '도봉',
 '노원',
 '은평',
 '서대문',
 '마포',
 '양천',
 '강서',
 '구로',
 '금천',
 '영등포',
 '동작',
 '관악',
 '서초',
 '강남',
 '송파',
 '강동',
 '중구',
 '서구',
 '동구',
 '영도',
 '부산진',
 '동래',
 '남구',
 '북구',
 '해운대',
 '기장',
 '사하',
 '금정',
 '강서',
 '연제',
 '수영',
 '사상',
 '중구',
 '동구',
 '서구',
 '남구',
 '북구',
 '수성',
 '달서',
 '달성',
 '중구',
 '동구',
 '남구',
 '연수',
 '남동',
 '부평',
 '계양',
 '서구',
 '강화',
 '옹진',
 '동구',
 '서구',
 '남구',
 '북구',
 '광산',
 '동구',
 '중구',
 '서구',
 '유성',
 '대덕',
 '중구',
 '남구',
 '동구',
 '북구',
 '울주',
 '세종특별자치',
 '수원 장안',
 '수원 권선',
 '수원 팔달',
 '수원 영통',
 '성남 수정',
 '성남 중원',
 '성남 분당',
 '의정부',
 '안양 만안',
 '안양 동안',
 '부천',
 '광명',
 '평택',
 '양주',
 '동두천',
 '안산 상록',
 '안산 단원',
 '고양 덕양',
 '고양 일산동',
 '고양 일산서',
 '과천',
 '의왕',
 '구리',
 '남양주',
 '오산',
 '화성',
 '시흥',
 '군포',
 '하남',
 '파주',
 '여주',
 '이천',
 '용인 처인',
 '용인 수지',
 '용인 기흥',
 '안성',
 '김포',
 '광주',
 '포천',
 '연천',
 '양평',
 '가평',
 '춘천',
 '원주',
 '강릉',
 '동해',
 '삼척',
 '태백',
 '정선',
 '속초',
 '고성',
 '양양',
 '인제',
 '홍천',
 '횡성',
 '영월',
 '평창',
 '화천',
 '양구

In [30]:
ID_candi = [sido_candi[n] + ' ' + sigun_candi[n] for n in range(0, len(sigun_candi))]

# 첫 글자가 띄어쓰기인 경우 정리하게끔 한다.
ID_candi = [name[1:] if name[0] == ' ' else name for name in ID_candi]

# 세종시는 예외처리
ID_candi = [name[:2] if name[:2] == '세종' else name for name in ID_candi]

ID_candi

['서울 종로',
 '서울 중구',
 '서울 용산',
 '서울 성동',
 '서울 광진',
 '서울 동대문',
 '서울 중랑',
 '서울 성북',
 '서울 강북',
 '서울 도봉',
 '서울 노원',
 '서울 은평',
 '서울 서대문',
 '서울 마포',
 '서울 양천',
 '서울 강서',
 '서울 구로',
 '서울 금천',
 '서울 영등포',
 '서울 동작',
 '서울 관악',
 '서울 서초',
 '서울 강남',
 '서울 송파',
 '서울 강동',
 '부산 중구',
 '부산 서구',
 '부산 동구',
 '부산 영도',
 '부산 부산진',
 '부산 동래',
 '부산 남구',
 '부산 북구',
 '부산 해운대',
 '부산 기장',
 '부산 사하',
 '부산 금정',
 '부산 강서',
 '부산 연제',
 '부산 수영',
 '부산 사상',
 '대구 중구',
 '대구 동구',
 '대구 서구',
 '대구 남구',
 '대구 북구',
 '대구 수성',
 '대구 달서',
 '대구 달성',
 '인천 중구',
 '인천 동구',
 '인천 남구',
 '인천 연수',
 '인천 남동',
 '인천 부평',
 '인천 계양',
 '인천 서구',
 '인천 강화',
 '인천 옹진',
 '광주 동구',
 '광주 서구',
 '광주 남구',
 '광주 북구',
 '광주 광산',
 '대전 동구',
 '대전 중구',
 '대전 서구',
 '대전 유성',
 '대전 대덕',
 '울산 중구',
 '울산 남구',
 '울산 동구',
 '울산 북구',
 '울산 울주',
 '세종',
 '수원 장안',
 '수원 권선',
 '수원 팔달',
 '수원 영통',
 '성남 수정',
 '성남 중원',
 '성남 분당',
 '의정부',
 '안양 만안',
 '안양 동안',
 '부천',
 '광명',
 '평택',
 '양주',
 '동두천',
 '안산 상록',
 '안산 단원',
 '고양 덕양',
 '고양 일산동',
 '고양 일산서',
 '과천',
 '의왕',
 '구리',
 '남양주',
 '오산',
 '화성',
 '시흥',
 '군포',
 '하남'

In [31]:
# 'ID'커럼을 추가하고 정제한 값들을 넣어준다.
election_result['ID'] = ID_candi
election_result.head(10)

,광역시도,시군,pop,moon,ahn,hong,ID
0,서울특별시,종로구,102566.0,42512.0,22313.0,22325.0,서울 종로
1,서울특별시,중구,82852.0,34062.0,19372.0,17901.0,서울 중구
2,서울특별시,용산구,148157.0,58081.0,32109.0,35230.0,서울 용산
3,서울특별시,성동구,203175.0,86686.0,45674.0,40566.0,서울 성동
4,서울특별시,광진구,240030.0,105512.0,52824.0,46368.0,서울 광진
5,서울특별시,동대문구,236092.0,98958.0,53359.0,51631.0,서울 동대문
6,서울특별시,중랑구,265706.0,111450.0,62778.0,56545.0,서울 중랑
7,서울특별시,성북구,295866.0,129263.0,66518.0,57584.0,서울 성북
8,서울특별시,강북구,210614.0,89645.0,51669.0,42268.0,서울 강북
9,서울특별시,도봉구,229233.0,94898.0,55600.0,47461.0,서울 도봉


In [32]:
# 각 후보들의 득표율을 계산해서 넣어준다.
election_result[['rate_moon', 'rate_hong', 'rate_ahn']] = election_result[['moon','hong','ahn']].div(election_result['pop'], axis = 0)
election_result[['rate_moon','rate_hong','rate_ahn']] *= 100
election_result.head()

,광역시도,시군,pop,moon,ahn,hong,ID,rate_moon,rate_hong,rate_ahn
0,서울특별시,종로구,102566.0,42512.0,22313.0,22325.0,서울 종로,41.448433,21.766472,21.754773
1,서울특별시,중구,82852.0,34062.0,19372.0,17901.0,서울 중구,41.111862,21.605996,23.381451
2,서울특별시,용산구,148157.0,58081.0,32109.0,35230.0,서울 용산,39.202333,23.778829,21.672280
3,서울특별시,성동구,203175.0,86686.0,45674.0,40566.0,서울 성동,42.665682,19.966039,22.480128
4,서울특별시,광진구,240030.0,105512.0,52824.0,46368.0,서울 광진,43.957839,19.317585,22.007249


In [34]:
# 문재인 후보의 득표율을 내림차순으로 보여준다.
election_result.sort_values(['rate_moon'], ascending = [False]).head(10)

,광역시도,시군,pop,moon,ahn,hong,ID,rate_moon,rate_hong,rate_ahn
63,광주광역시,광산구,248209.0,159119.0,65402.0,3630.0,광주 광산,64.106862,1.462477,26.349568
188,전라남도,구례군,18924.0,11873.0,5188.0,628.0,구례,62.740435,3.318537,27.414923
62,광주광역시,북구,292001.0,177850.0,88501.0,4241.0,광주 북구,60.907326,1.452392,30.308458
60,광주광역시,서구,204787.0,124183.0,62218.0,3182.0,광주 서구,60.640080,1.553810,30.381811
194,전라남도,완도군,34886.0,20945.0,11021.0,762.0,완도,60.038411,2.184257,31.591469
191,전라남도,화순군,43572.0,25867.0,13832.0,770.0,화순,59.366107,1.767190,31.745157
61,광주광역시,남구,146037.0,85642.0,47663.0,2521.0,광주 남구,58.644042,1.726275,32.637619
200,전라남도,함평군,23368.0,13539.0,7831.0,471.0,함평,57.938206,2.015577,33.511640
187,전라남도,곡성군,21247.0,12152.0,6850.0,624.0,곡성,57.193957,2.936885,32.239846
189,전라남도,고흥군,47036.0,26882.0,15614.0,1349.0,고흥,57.151969,2.868016,33.195850


In [35]:
# 홍준표 후보의 득표율을 내림차순으로 보여준다.
election_result.sort_values(['rate_hong'], ascending = [False]).head(10)

,광역시도,시군,pop,moon,ahn,hong,ID,rate_moon,rate_hong,rate_ahn
219,경상북도,군위군,17627.0,2251.0,1939.0,11651.0,군위,12.770182,66.097464,11.000170
220,경상북도,의성군,37855.0,5365.0,4767.0,23790.0,의성,14.172500,62.845067,12.592788
223,경상북도,영덕군,26125.0,3786.0,3231.0,16314.0,영덕,14.491866,62.445933,12.367464
215,경상북도,청도군,30398.0,5323.0,3654.0,17678.0,청도,17.511020,58.155142,12.020528
240,경상남도,창녕군,42878.0,10310.0,3877.0,24464.0,창녕,24.044965,57.054900,9.041933
212,경상북도,문경시,49113.0,8616.0,6905.0,27832.0,문경,17.543217,56.669314,14.059414
211,경상북도,상주시,67717.0,12356.0,9392.0,38171.0,상주,18.246526,56.368416,13.869486
224,경상북도,봉화군,22947.0,4209.0,3370.0,12536.0,봉화,18.342267,54.630235,14.686016
222,경상북도,영양군,12048.0,2281.0,1701.0,6554.0,영양,18.932603,54.399070,14.118526
210,경상북도,영천시,66091.0,12250.0,9175.0,35328.0,영천,18.535050,53.453572,13.882374


In [36]:
# 안철수 후보의 득표율을 내림차순으로 보여준다.
election_result.sort_values(['rate_ahn'], ascending = [False]).head(10)

,광역시도,시군,pop,moon,ahn,hong,ID,rate_moon,rate_hong,rate_ahn
196,전라남도,진도군,21189.0,10392.0,8855.0,511.0,진도,49.044315,2.411629,41.790552
201,전라남도,신안군,28950.0,14370.0,12000.0,713.0,신안,49.637306,2.462867,41.450777
197,전라남도,영암군,36402.0,18999.0,13610.0,825.0,영암,52.192187,2.266359,37.388056
59,광주광역시,동구,66287.0,37053.0,23438.0,1308.0,광주 동구,55.897838,1.973238,35.358366
192,전라남도,장흥군,27149.0,14821.0,9593.0,636.0,장흥,54.591329,2.342628,35.334635
198,전라남도,무안군,52516.0,29516.0,18052.0,983.0,무안,56.203824,1.871810,34.374286
200,전라남도,함평군,23368.0,13539.0,7831.0,471.0,함평,57.938206,2.015577,33.511640
189,전라남도,고흥군,47036.0,26882.0,15614.0,1349.0,고흥,57.151969,2.868016,33.195850
61,광주광역시,남구,146037.0,85642.0,47663.0,2521.0,광주 남구,58.644042,1.726275,32.637619
187,전라남도,곡성군,21247.0,12152.0,6850.0,624.0,곡성,57.193957,2.936885,32.239846


In [38]:
# csv 파일을 읽어온다. -> 5장에서 실습과정 중 생기는 거 같은데 아직 5장 실습이 안 끝남.
draw_korea = pd.read_csv('data/05. draw_korea.csv', encoding='utf-8', index_col = 0)
draw_korea.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/05. draw_korea.csv'

In [ ]:
# draw_korea에 있는 지역에서 election_result에 있는 지역을 빼준다.
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

In [ ]:
# 위의 것의 반대로(서로 없는 지역을 알 수 있다.)
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

In [ ]:
# '고성'지역을 찾아낸다.
election_result[election_result['ID'] == '고성']

In [ ]:
# 강원/경남 고성을 각각 나누어 준다.
election_result.loc[125, 'ID'] = '고성(강원)'
election_result.loc[233, 'ID'] = '고성(경남)'

election_result[election_result['시군'] == '고성군']

In [39]:
# 경상남도 지역을 찾아낸다.
election_result[election_result['광역시도'] == '경상남도']

,광역시도,시군,pop,moon,ahn,hong,ID,rate_moon,rate_hong,rate_ahn
226,경상남도,창원시의창구,164047.0,60757.0,22830.0,56887.0,창원 의창,37.036337,34.677257,13.916743
227,경상남도,창원시성산구,153327.0,63717.0,22923.0,42052.0,창원 성산,41.556282,27.426350,14.950400
228,경상남도,창원시마산합포구,119281.0,35592.0,14686.0,54488.0,창원 마산합포,29.838784,45.680368,12.312103
229,경상남도,창원시마산회원구,NaN,NaN,NaN,NaN,창원 마산회원,NaN,NaN,NaN
230,경상남도,창원시진해구,114779.0,41249.0,17435.0,40049.0,창원 진해,35.937759,34.892271,15.190061
231,경상남도,진주시,222813.0,73929.0,26687.0,93751.0,진주,33.179841,42.076091,11.977308
232,경상남도,통영시,82855.0,25477.0,10738.0,36128.0,통영,30.748899,43.603886,12.959990
233,경상남도,고성군,34603.0,9848.0,4104.0,16797.0,고성,28.459960,48.542034,11.860243
234,경상남도,사천시,NaN,NaN,NaN,NaN,사천,NaN,NaN,NaN
235,경상남도,김해시,318253.0,147972.0,45126.0,82880.0,김해,46.495084,26.042174,14.179285


In [ ]:
# 마산합포구를 합포, 회원으로 변경
election_result.loc[228, 'ID'] = '창원 합포'
election_result.loc[229, 'ID'] = '창원 회원'

election_result[election_result['광역시도'] == '경상남도']

In [ ]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

In [ ]:
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

In [ ]:
# 부턴시를 찾아낸다.
election_result[election_result['시군'] == '부천시']

In [ ]:
# 데이터 끝부분을 출력 <> head()
election_result.tail()

In [40]:
# 부천시의 득표율을 알기 위해서, 후보가 3명이니 3으로 나누어준다.
ahn_tmp  = election_result.loc[85, 'ahn'] / 3
hong_tmp = election_result.loc[85, 'hong'] / 3
moon_tmp = election_result.loc[85, 'moon'] / 3
pop_tmp  = election_result.loc[85, 'pop'] / 3

rate_moon_tmp = election_result.loc[85, 'rate_moon']
rate_hong_tmp = election_result.loc[85, 'rate_hong']
rate_ahn_tmp = election_result.loc[85, 'rate_ahn']

# 위에서 부천시의 지역을 수정해주던 것을 마루리한다.
election_result.loc[250] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp, 
                           '경기도','부천시','부천 소사',
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]
election_result.loc[251] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp, 
                           '경기도','부천시','부천 오정',
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]
election_result.loc[252] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp, 
                           '경기도','부천시','부천 원미',
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]

In [41]:
election_result[election_result['시군']=='부천시']

,광역시도,시군,pop,moon,ahn,hong,ID,rate_moon,rate_hong,rate_ahn
85,경기도,부천시,NaN,NaN,NaN,NaN,부천,NaN,NaN,NaN


In [ ]:
# 실제 데이터에도 적용
election_result.drop([85] , inplace = True)
election_result[election_result['시군'] == '부천시']

In [ ]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

In [ ]:
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

In [ ]:
# 두 데이터를 합쳐준다.
final_elect_data = pd.merge(election_result, draw_korea, how='left', on=['ID'])
final_elect_data.head()

In [ ]:
# 각 후보들의 득표율을 비교한다.
final_elect_data['moon_vs_hong'] = final_elect_data['rate_moon'] - final_elect_data['rate_hong']
final_elect_data['moon_vs_ahn'] = final_elect_data['rate_moon'] - final_elect_data['rate_ahn']
final_elect_data['ahn_vs_hong'] = final_elect_data['rate_ahn'] - final_elect_data['rate_hong']
final_elect_data.head()

## 6-4. 19대 대선 결과 득표율 시각화하기 

In [43]:
# 지역별로 경계선을 나누어준다.
BORDER_LINES = [
    [(5,1), (5,2), (7,2), (7,3), (11,3), (11,0)], # 인천
    [(5,4), (5,5), (2,5), (2,7), (4,7), (4,9), (7,9),
     (7,7), (9,7), (9,5), (10,5), (10,4), (5,4)], # 서울
    [(1,7), (1,8), (3,8), (3,10), (10,10), (10,7),
    (12,7), (12,6), (11,6), (11,5), (12,5), (12,4),
    (11,4), (11,3)], # 경기도
    [(8,10), (8,11), (6,11), (6,12)], # 강원도
    [(12,5), (13,5), (13,4), (14,4), (14,5), (15,5),
    (15,4), (16,4), (16,2)], # 충청북도
    [(16,4), (17,4), (17,5), (16,5), (16,6), (19,6),
    (19,5), (20,5), (20,4), (21,4), (21,3), (19,3), (19,1)], # 전라북도
    [(13, 5), (13,6), (16,6)], # 대전시
    [(13,5), (14,5)], # 세종시
    [(21,2), (21,3), (22,3), (22,4), (24,4), (24,2), (21,2)], # 광주
    [(20,5), (21,5), (21,6), (23,6)], # 전라남도
    [(10,8), (12,8), (12,9), (14,9), (14,8), (16,8), (16,6)], # 충천북도
    [(14,9), (14,11), (14,12), (13,12), (13,13)], # 경상북도
    [(15,8), (17,8), (17,10), (16,10), (16,11), (14,11)], # 대구
    [(17,9), (18,9), (18,8), (19,8), (19,9), (20,9), (20,10), (21,10)], # 부산
    [(16,11), (16,13)], # 울산
    [(27, 5), (27,6), (25,6)],
]

In [46]:
def drawKorea(targetData, blockedMap, cmapname) :
    gamma = 0.75
    
    whitelabelmin = 20
    
    datalabel = targetData
    
    # 블록모양의 그래프 사용
    tmp_max = max([ np.abs(min(blockedMap[targetData])), np.abs(max(blockedMap[targetData]))])
    vmin, vmax = -tmp_max, tmp_max
    
    mapdata = blockedMap.pivot_table(index = 'y', columns = 'x', values=targetData)
    masked_mapdata = np.ma.masked_where(np.isnan(mapdata), mapdata)
    
    plt.figure(figsuze = (9, 11))
    plt.pcolor(masked_mapdata, vmin=vmin, vmax=vmax, cmap=cmapname, edgecolor='#aaaaaa', linewidth = 0.5)
    
    # 지역 이름 표시
    for idx, row in blockedMap, iterrows() :
        # 광역시는 구 이름이 겹치는 경우가 많아서 시단위 이름도 같이 표시한다.
        # (중구, 서구)
        if len(row['ID'].split()) == 2:
            dispname = '{}\n{}'.format(row['ID'].split()[0], row['ID'].split()[1])
        elif row['ID'][:2] == '고성' :
            dispname = '고성'
        else :
            dispname = row['ID']

        # 서대문구, 서귀포시 같이 이름이 3자 이상인 경우에 작은 글자로 표시한다.
        if len(dispname.splitlines()[-1]) >= 3:
            fontsize, linespacing = 10.0, 1.1
        else :
            fontsize, linespacing = 11, 1.
            
        annocolor = 'white' if np.abs(row[targetData]) > whitelabelmin else 'black'
        plt.annotate(dispname, (row['x'] + 0.5, row['y'] + 0.5), weight='bold',
                     fontsize = fontsize, ha = 'center', va = 'center', color = annocolor,
                     linespacing = linespacing)

    # 시도 경계 그린다.
    for path in BORDER_LINES :
        ys, xs = zip(*path)
        plt.plot(xs, ys, c = 'black', lw = 2)
    
    plt.gca().invert_yaxis()
    
    plt.axis('off')
    
    cb = plt.colorbar(shrink = .1, aspect = 10)
    cb.set_label(datalabel)
    
    plt.tight_layout()
    plt.show()

In [47]:
# 문재인과 홍준표 후보의 득표율 비교
drawKorea('moon_vs_hong', final_elect_data, 'RdBu')

NameError: name 'final_elect_data' is not defined

In [ ]:
# 문재인과 안철수 후보의 득표율 비교
drawKorea('moon_vs_ahn', final_elect_data, 'RdBu')

In [ ]:
# 안철수와 홍준표 후보의 득표율 비교
drawKorea('ahn_vs_hong', final_elect_data, 'RdBu')

In [ ]:
import folium
import json
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [ ]:
# ID를 index로 잡는다.
pop_folium = final_elect_data.set_index('ID')

# 광역시도, 시군을 삭제
del pop_folium['광역시도']
del pop_folium['시군']

pop_folium.head()

In [ ]:
# json파일을 불러온다
geo_path = 'data/05. skorea_municipalities_geo_simple.json'
geo_data = json.load(open(geo_path, encoding = 'utf-8'))

# 문제인과 홍준표 후보의 득표율을 비교한다.
map = folium.Map(location = [36.2002, 127.054], zoom_start = 6)
map.choropleth(geo_str = geo_str,
              data = pop_folium['moon_vs_hong'],
              columns = [pop_folium.index, pop_folium['moon_vs_hong']],
              fill_color = 'PuBu', # PuRd, YlGnBu
              key_on = 'feature.id')

map